In [1]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/senior-data-analyst-itc/job/R-61809")
page_data = loader.load().pop().page_content
print(page_data)






















Senior Data Analyst, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



S

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Data Analyst, ITC',
 'experience': '5+ years of experience in data and analytics space with hands-on experience',
 'skills': ['SQL', 'Big Data', 'PySpark', 'Databricks', 'Hive', 'Snowflake'],
 'description': "The Senior Data Analyst will work with the Data and Artificial Intelligence team at Nike. This role is crucial in translating the business needs of Nike into data requirements and thereby have a significant impact on the growth of Nike's business."}

In [10]:
type(json_res)

dict

In [11]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [14]:
job = json_res

In [15]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [16]:
job

{'role': 'Senior Data Analyst, ITC',
 'experience': '5+ years of experience in data and analytics space with hands-on experience',
 'skills': ['SQL', 'Big Data', 'PySpark', 'Databricks', 'Hive', 'Snowflake'],
 'description': "The Senior Data Analyst will work with the Data and Artificial Intelligence team at Nike. This role is crucial in translating the business needs of Nike into data requirements and thereby have a significant impact on the growth of Nike's business."}

In [17]:
job = json_res
job['skills']

['SQL', 'Big Data', 'PySpark', 'Databricks', 'Hive', 'Snowflake']

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Rahul, a business development executive at ABC. ABC is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ABC 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase ABC's portfolio: {link_list}
        Remember you are Rahul, BDE at ABC. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Empowering Nike's Data-Driven Growth with ABC's Expertise

Dear Hiring Manager,

I came across the Senior Data Analyst, ITC role at Nike and was impressed by the company's commitment to leveraging data and analytics to drive business growth. As a business development executive at ABC, an AI & Software Consulting company, I believe our expertise can help Nike achieve its goals.

With over 5+ years of experience in the data and analytics space, our team at ABC has hands-on experience with SQL, Big Data, PySpark, Databricks, Hive, and Snowflake. We have a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in data analytics can help Nike translate business needs into data requirements, ultimately driving significant impact on the company's growth.

I'd like to highlight some of our notable projects that demonstrate our capabilities:

* Our work in machine learning and 